In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import OneHotEncoder
from scipy import stats
import seaborn as sns

resultado_anterior = 0
maximo = 0

In [12]:
entrenar_modelo({})

In [11]:
def guardar_submit(params, result):
    tiempo = "time"
    with open("historial_submits.txt","a+") as f:
        f.write("\n"+tiempo+"|"+params+"|")

In [3]:
def entrenar_modelo(params):
    result = None#entrenar(params)
    guardar_submit(params, result)
    

In [ ]:
def modelfit(model, features, labels, usarCrossValidation=True, cv_folds=5, early_stopping_rounds=50):
    if usarCrossValidation:
        parametros = model.get_xgb_params()
        dmatrix = xgb.DMatrix(features.values, label=labels.values)
        cvresult = xgb.cv(parametros, dmatrix, num_boost_round=model.get_params()['n_estimators'], nfold=cv_folds,
                          metrics='auc', #verbose_eval=True,
                          early_stopping_rounds=early_stopping_rounds)
        model.set_params(n_estimators=cvresult.shape[0])
    
    model.fit(features, labels,eval_metric='auc')

In [ ]:
X, y = dfPredictorLocal.iloc[:,:-1], dfPredictorLocal.iloc[:,-1:]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

modelo = xgb.XGBRegressor(
 #booster='dart',
 learning_rate =0.01,
 n_estimators=1000,
 max_depth=4,
 min_child_weight=8,
 gamma=0.22,
 subsample=1,
 colsample_bytree=1,
 objective= 'binary:logistic',
 nthread=-1,
 scale_pos_weight=18.8,

 #sample_type='weighted',
 #rate_drop=0.1,
 #skip_dropout=0.5,
 random_state=272)

modelfit(modelo, X_train, y_train, early_stopping_rounds=30)

predsa = modelo.predict(X_test)
resultado = roc_auc_score(y_test, predsa)
print(resultado)
print("Vario: {} con respecto al anterior".format(resultado-resultado_anterior))
if (maximo == resultado):
    print(" Maximo igualado")
else:
    print("Maximo anterior {}".format(maximo))
if resultado>maximo:
    print("Nuevo MAXIMO : {}".format(resultado-maximo))
    maximo = resultado
resultado_anterior = resultado